In [1]:
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import numpy as np
import keras
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, AveragePooling2D, Activation, GlobalAveragePooling2D, Lambda
from keras.layers.core import Dropout
from keras.layers.merge import Concatenate
from keras.layers.normalization import BatchNormalization
from keras.initializers import Initializer
from keras.regularizers import l2
from keras.models import Model
from keras.optimizers import Adam
from PIL import Image
import scipy.misc
import matplotlib.pyplot as plt
from scipy.misc import imsave
import pydot
import graphviz
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
(xx_train, yy_train), (x_test, y_test) = cifar10.load_data()
x_train = xx_train[:40000]
y_train = yy_train[:40000]
x_validate = xx_train[40000:50000]
y_validate = yy_train[40000:50000]

x_train = x_train/255.0
x_validate = x_validate/255.0
x_test = x_test/255.0

In [3]:
def dense_block(x,num_layers_per_block,growth_rate):
    for i in range(num_layers_per_block//2):
        x_ = BatchNormalization()(x)
        x_ = Activation('relu')(x_)
        x_ = Conv2D(number_filters,(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001))(x_)
        x_ = Dropout(0.2)(x_)
        x_ = BatchNormalization()(x_)
        x_ = Activation('relu')(x_)
        x = Concatenate()([x,x_])
    return x 

def transition_layers(x,compression):
    updated_num_filters = int(x.get_shape().as_list()[-1] * compression)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(updated_num_filters,(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001))(x)
    x = Dropout(0.2)(x)
    x = AveragePooling2D()(x)
    return x

In [4]:
#Define model parameters
model_depth = 100
num_dense_blocks = 3
growth_rate = 12
number_filters = 16
compression = 0.5
num_layers_per_block = (model_depth - 4) // num_dense_blocks

In [5]:
#Let's define the model
inp = Input(x_train.shape[1:])
x = Conv2D(number_filters,(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001))(inp)
for i in range(num_dense_blocks):
    x = dense_block(x,num_layers_per_block,growth_rate)
    if (i != num_dense_blocks-1):
        x = transition_layers(x,compression)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001))(x)

model = Model(inp, x)
model.compile(Adam(), loss='sparse_categorical_crossentropy',metrics=['accuracy'])
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_acc', patience=15, verbose=0, mode='auto')
plot_model(model, to_file='model.png')

In [6]:
model.fit(x_train, y_train, batch_size=128, epochs=120,shuffle=True,
          validation_data=(x_validate, y_validate),callbacks=[earlyStopping])

Train on 40000 samples, validate on 10000 samples
Epoch 1/120
40000/40000 [==============================] - 181s - loss: 1.5732 - acc: 0.4924 - val_loss: 1.8264 - val_acc: 0.4346
Epoch 2/120
40000/40000 [==============================] - 174s - loss: 1.1432 - acc: 0.6556 - val_loss: 1.4551 - val_acc: 0.5880
Epoch 3/120
40000/40000 [==============================] - 175s - loss: 0.9696 - acc: 0.7214 - val_loss: 1.1464 - val_acc: 0.6629
Epoch 4/120
40000/40000 [==============================] - 175s - loss: 0.8473 - acc: 0.7708 - val_loss: 0.9254 - val_acc: 0.7528
Epoch 5/120
40000/40000 [==============================] - 175s - loss: 0.7693 - acc: 0.8014 - val_loss: 0.8630 - val_acc: 0.7672
Epoch 6/120
40000/40000 [==============================] - 175s - loss: 0.7058 - acc: 0.8250 - val_loss: 1.1305 - val_acc: 0.6971
Epoch 7/120
40000/40000 [==============================] - 175s - loss: 0.6682 - acc: 0.8401 - val_loss: 1.1043 - val_acc: 0.7034
Epoch 8/120
40000/40000 [===============

40000/40000 [==============================] - 174s - loss: 0.3941 - acc: 0.9571 - val_loss: 0.7222 - val_acc: 0.8735
Epoch 64/120
40000/40000 [==============================] - 174s - loss: 0.3978 - acc: 0.9554 - val_loss: 0.8464 - val_acc: 0.8492
Epoch 65/120
40000/40000 [==============================] - 174s - loss: 0.3913 - acc: 0.9581 - val_loss: 0.8844 - val_acc: 0.8450
Epoch 66/120
40000/40000 [==============================] - 174s - loss: 0.3934 - acc: 0.9580 - val_loss: 0.8630 - val_acc: 0.8480
Epoch 67/120
40000/40000 [==============================] - 175s - loss: 0.3924 - acc: 0.9571 - val_loss: 0.8377 - val_acc: 0.8596
Epoch 68/120
40000/40000 [==============================] - 174s - loss: 0.3809 - acc: 0.9619 - val_loss: 0.7765 - val_acc: 0.8655
Epoch 69/120
40000/40000 [==============================] - 174s - loss: 0.3797 - acc: 0.9616 - val_loss: 0.8663 - val_acc: 0.8581
Epoch 70/120
40000/40000 [==============================] - 174s - loss: 0.3888 - acc: 0.9571 - 

In [7]:
model.save_weights('densenet100_cifar10_120epochs.h5')

In [8]:
evaluation = model.evaluate(x_test,y_test, batch_size=512, verbose=1)
print('Loss: %.2f Accuracy: %.2f%%' % (evaluation[0], evaluation[1]*100.0))

10000/10000 [==============================] - 16s    
Loss: 0.90 Accuracy: 84.46%
